In [ ]:
# 统计数据中level的分布情况
import json

with open('hotpotqa.json', 'r') as f:
    data = json.load(f)

level = {}
for item in data:
    if item['level'] in level:
        level[item['level']] += 1
    else:
        level[item['level']] = 1

print(level)

In [ ]:
import json

# sample 100 hard question from hotpotqa

with open('hotpot.jsonl') as f:
    data = [json.loads(line) for line in f]

# hard_data = [d for d in data if d['level'] == 'hard']
# # shuffle
# import random
# random.shuffle(hard_data)

# permute the order of the questions, give new id
# for i, d in enumerate(hard_data):
    # d['_id'] = i

# with open('hotpot_100_hard.jsonl', 'w') as f:
#     for d in hard_data[:1000]:
#         f.write(json.dumps(d) + '\n')

# sample 1000 question from hotpotqa

with open('hotpot_1000.jsonl', 'w') as f:
    for d in data[:1000]:
        f.write(json.dumps(d) + '\n')

In [ ]:
import json

with open('hotpotqa.json') as f:
    data = json.load(f)

# read first 1000 questions from hotpotqa and rearrange the id
for i, d in enumerate(data[:100]):
    d['_id'] = i

# store as a jsonl file
with open('hotpotqa_100.jsonl', 'w') as f:
    for d in data[:100]:
        f.write(json.dumps(d) + '\n')



In [4]:
import json
from collections import defaultdict

def analyze_jsonl(file_path):
    task_ids = defaultdict(int)
    max_id = -1
    problematic_lines = []

    # Read the file and count occurrences of task_ids
    with open(file_path, 'r') as file:
        for line_number, line in enumerate(file, 1):
            try:
                data = json.loads(line.strip())
                task_id = data['task_id']
                task_ids[task_id] += 1
                max_id = max(max_id, task_id)
            except json.JSONDecodeError as e:
                problematic_lines.append((line_number, line, str(e)))
            except KeyError:
                problematic_lines.append((line_number, line, "Missing 'task_id' key"))

    # Find repetitive task_ids
    repetitive_ids = [id for id, count in task_ids.items() if count > 1]

    # Find missing task_ids
    all_ids = set(range(1000))  # 0 to 999
    existing_ids = set(task_ids.keys())
    missing_ids = list(all_ids - existing_ids)

    return repetitive_ids, missing_ids, max_id, problematic_lines

# Usage
file_path = "samples_1000_sup_unique.jsonl"  # Replace with your actual file path
repetitive_ids, missing_ids, max_id, problematic_lines = analyze_jsonl(file_path)

print(f"Repetitive task_ids: {repetitive_ids}")
print(f"Missing task_ids: {missing_ids}")
print(f"Maximum task_id found: {max_id}")

if problematic_lines:
    print("\nProblematic lines:")
    for line_number, line, error in problematic_lines:
        print(f"Line {line_number}: {error}")
        print(f"Content: {line.strip()[:100]}...")  # Print first 100 characters of the line
        print()

Repetitive task_ids: []
Missing task_ids: [528, 707, 582, 967, 974, 849, 722, 725, 726, 599, 854, 984, 994, 745, 753, 881, 758, 759, 767]
Maximum task_id found: 999


In [ ]:
# delete the repetitive task_ids only save one
import json
from collections import defaultdict

def remove_repetitive_task_ids(file_path):
    task_ids = defaultdict(list)
    lines_to_keep = []

    # Read the file and store the lines for each task_id
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line.strip())
            task_id = data['task_id']
            task_ids[task_id].append(line)

    # Keep the first line for each task_id
    for lines in task_ids.values():
        lines_to_keep.append(lines[0])

    # Write the lines to a new file
    new_file_path = file_path.replace(".jsonl", "_unique.jsonl")
    with open(new_file_path, 'w') as file:
        for line in lines_to_keep:
            file.write(line)

    return new_file_path

# Usage
file_path = "samples_1000_sup.jsonl"  # Replace with your actual file path
new_file_path = remove_repetitive_task_ids(file_path)